## A Column Generation Example -- Metal Cutting

Suppose we stock standard-length solid metal blocks that are used to craft android robots. The stock length is 19 meters.

We have an order for

- 12 lengths of 4m
- 15 lengths of 5m
- 22 lengths of 6m


How should we cut our standard length metal blocks to meet the demand and minimize the number of standard lengths we need to use?

In [3]:
using JuMP, Gurobi

scale = 10 # factor to increase/decrease size of problem

N = 16*scale  # upper bound on number of pipes needed

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m,"OutputFlag",0)

# times to cut each pattern 1,2,3 from given block 1 ~ N
@variable(m, x[1:3, 1:N] >= 0, Int)

# use block 1 ~ N
@variable(m, z[1:N], Bin)

for j = 1:N
    # only use patterns that fit on the block
    @constraint(m, 4 * x[1,j] + 5 * x[2,j] + 6 * x[3,j] <= 19)
end

# met demand for pattern 1
@constraint(m, sum(x[1,j] for j=1:N) >= 12 * scale)
# met demand for pattern 2
@constraint(m, sum(x[2,j] for j=1:N) >= 15 * scale)
# met demand for pattern 3
@constraint(m, sum(x[3,j] for j=1:N) >= 22 * scale)

for j = 1:N
    # upper bounds on how many of each pattern can be used (0 if z = 0)
    @constraint(m, x[1,j] <= 4 * z[j])
    @constraint(m, x[2,j] <= 3 * z[j])
    @constraint(m, x[3,j] <= 3 * z[j])
end

# symmetry-breaking
for j = 1:N-1
    @constraint(m, z[j] >= z[j+1])
end

# minimize the total blocks used
@objective(m, Min, sum(z[j] for j=1:N))

@time(optimize!(m))

Academic license - for non-commercial use only - expires 2022-07-06
  1.582436 seconds (10.52 k allocations: 1.215 MiB)


In [4]:
x

3×160 Matrix{VariableRef}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]  …  x[1,158]  x[1,159]  x[1,160]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]     x[2,158]  x[2,159]  x[2,160]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]     x[3,158]  x[3,159]  x[3,160]

In [5]:
value.(x)

3×160 Matrix{Float64}:
 -0.0   1.0  2.0  -0.0  -0.0  -0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -0.0   3.0  1.0  -0.0  -0.0  -0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  3.0  -0.0  1.0   3.0   3.0   3.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [ ]:
4 * x[1,j]

In [6]:
z

160-element Vector{VariableRef}:
 z[1]
 z[2]
 z[3]
 z[4]
 z[5]
 z[6]
 z[7]
 z[8]
 z[9]
 z[10]
 z[11]
 z[12]
 z[13]
 ⋮
 z[149]
 z[150]
 z[151]
 z[152]
 z[153]
 z[154]
 z[155]
 z[156]
 z[157]
 z[158]
 z[159]
 z[160]

In [7]:
value.(z)

160-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

## Second Modeling Approach (Column Generation)

In [3]:
using JuMP, Gurobi

# all the columns:
A = [0 0 1 0 2 1 2 3 4
     0 1 0 2 1 2 2 1 0
     3 2 2 1 1 0 0 0 0]

scale = 1000000000

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

@variable(m, x[1:9] >= 0, Int)

@constraint(m, A*x .>= [12;15;22]*scale)

@objective(m, Min, sum(x))

@time(optimize!(m))

Academic license - for non-commercial use only - expires 2022-07-06
  0.003687 seconds (771 allocations: 61.883 KiB)
